In [ ]:
!pip install datasets transformers

In [ ]:
import pandas as pd
import os
import requests
import time
import sys
import json
from requests_oauthlib import OAuth1Session
from requests_oauthlib import OAuth1
import base64
from collections import Counter, defaultdict
import transformers
import random
import datasets
import tokenize
import io
import re
import time
import math
import datetime

# Data Collect

In [ ]:
# GitHub Access Token
with open('github_token', 'r') as accestoken:
    access = accestoken.readline().replace("\n","")

In [ ]:
%run -i commits_from_GitHub.py

In [ ]:
%run -i commit_diffs.py

In [ ]:
%run -i diff_commit_data.py

# Data Preprocessing

## Imports and Functions

In [ ]:
def tag_vulparts(token_list, mode, tester):
    ner_tag_dict_nums = {'rce':[1, 2],
                         'oob':[3, 4],
                         'xss':[5, 6],
                         'sql':[7, 8],
                         'iiv':[9,10],
                         'pat':[11,12],
                        }

    num1 = ner_tag_dict_nums[mode][0]
    num2 = ner_tag_dict_nums[mode][1]

    ner_tags = []
    for idx, tok in enumerate(token_list):
        if idx == tester[0]:
            ner_tags.append(num1)
        elif idx in range(tester[0], tester[1]+1):
            ner_tags.append(num2)
        else:
            ner_tags.append(0)

    return ner_tags

In [ ]:
def getneutralText(text):
    newtext = ''
    lines = text.split("\n")
    for line in lines:
        line = line.strip()
        if len(line) > 0:
            if line[0] == "-":
                continue
            elif line[0] == "+":
                newtext = newtext + '\n' + line[1:]
            else:
                newtext = newtext + '\n' + line
    return newtext

In [ ]:
def tag_vulparts_v2(token_list, mode, tester, token_dict):
    ner_tag_dict_nums = {'rce':[1, 2],
                         'oob':[3, 4],
                         'xss':[5, 6],
                         'sql':[7, 8],
                         'iiv':[9,10],
                         'pat':[11,12],
                        }

    num1 = ner_tag_dict_nums[mode][0]
    num2 = ner_tag_dict_nums[mode][1]
    word_ids = token_dict.word_ids()
    ner_tags = []
    i1s = []
    i1s_idx = []
    for idx, tok in enumerate(token_list):
        if idx == tester[0]:
            ner_tags.append(num1)
            b = word_ids[idx]
            b_idx = idx
        elif idx in range(tester[0]+1, tester[1]+1):
            ner_tags.append(num2)
            i1s.append(word_ids[idx])
            i1s_idx.append(idx)
        else:
            ner_tags.append(0)
    if b in i1s:
        for i, v in enumerate(i1s):
            if v == b:
                ner_tags[i1s_idx[i]] = num1
    return ner_tags

In [ ]:
def shorten_data_with_windows_v2(infodict):
    '''
    taking the really large samples and making them smaller
    '''
    new_info = []
    text = infodict['text']
    new_size = 3000

    if 'cdef ' in text[:50]:
      return []
    if len(text) > 500000:
      return []
    # neutral parts
    if infodict.type == 'gp':
      if len(text) >= new_size:
          chunk_start = 0
          chunk_size = new_size
          breaks = re.finditer(r"[\n]|$|[)]|[}]|[]]",text)
          break_ids = [m.end(0) for m in breaks]
          break_ids_to_use = []
          for i in range(math.ceil(len(text)/new_size)):
            if i == 0:
              break_id = [m for m in break_ids if m <=new_size][-1]
              chunk_start = 0
            else:
              break_id = [m for m in break_ids if m >= break_ids_to_use[i-1] and m <= break_ids_to_use[i-1]+new_size][-1]
              chunk_start = break_ids_to_use[i-1]
            break_ids_to_use.append(break_id)
            chunk = [chunk_start, break_id]
            new_tokens = text[chunk[0]:chunk[1]]
            new_info.append(new_tokens)

      else:
        ## If there are not more than 512 tokens
        new_tokens = text
        new_info.append(new_tokens)

    else: #vulparts
        bp_len = infodict['bp_len']
        bp_index = infodict['bp_index']+1
        bp_end = bp_index+bp_len-1
        tag_indeces = [bp_index, bp_end+1]

        # If there are more than 512 tokens
        if len(text) >= new_size:
          # If the indeces start and stop before 512, cut the list to 512
          if tag_indeces[0] < new_size and tag_indeces[1] <= new_size:
            breaks = re.finditer(r"[\n]|$|[)]|[}]|[]]",text)
            break_ids = [m.end(0) for m in breaks]
            break_id_to_use = [m for m in break_ids if m <= new_size][-1]
            new_tokens = text[:break_id_to_use]
            new_info.append(new_tokens)
          elif bp_len <= new_size:
            breaks = re.finditer(r"[\n]|$|[)]|[}]|[]]",text)
            break_ids = [m.end(0) for m in breaks]
            break_id_to_use1 = [m for m in break_ids if m <= bp_index][-1]
            break_id_to_use2 = [m for m in break_ids if m >= break_id_to_use1 and m <= break_id_to_use1+new_size][-1]
            new_tokens = text[break_id_to_use1:break_id_to_use2]
            new_info.append(new_tokens)
          else:
            breaks = re.finditer(r"[\n]|$|[)]|[}]|[]]",text)
            break_ids = [m.end(0) for m in breaks]
            break_ids_to_use = []
            for i in range(math.ceil(len(text)/new_size)):
              if i == 0:
                break_id = [m for m in break_ids if m <=new_size][-1]
                chunk_start = 0
              else:
                break_id = [m for m in break_ids if m >= break_ids_to_use[i-1] and m <= break_ids_to_use[i-1]+new_size][-1]
                chunk_start = break_ids_to_use[i-1]
              break_ids_to_use.append(break_id)
              chunk = [chunk_start, break_id]
              new_tokens = text[chunk[0]:chunk[1]]
              new_info.append(new_tokens)

        else:
            ## If there are not more than 512 tokens
            new_tokens = text
            new_info.append(new_tokens)

    return new_info

In [ ]:
def CODEBERT_tokenizer_tag_v2(row):
    mode = row.cwetype
    content = row.short_text

    new_info = []
    if 'cdef ' in content[:50]:
        return []
    if len(content) > 100000:
        return []

    token_dict = tokenizer(content.strip(), truncation=False)
    input_ids = token_dict['input_ids']
    attention_mask = token_dict['attention_mask']
    tokens = tokenizer.convert_ids_to_tokens(token_dict.input_ids)
    token_list = [tok for tok in tokens if tok != '<s>' and tok != '<pad>' and tok != '</s>']
    tok_results = []

    if row.type == 'bp':
        bp = row.parts

        bp_token_dict = tokenizer(bp.strip(), truncation = False)
        bp_tokens = tokenizer.convert_ids_to_tokens(bp_token_dict.input_ids)
        bp_token_list = [bp_tok for bp_tok in bp_tokens if bp_tok != '<s>' and bp_tok != '<pad>' and bp_tok != '</s>']

        if bp_token_list == [] or len(set(bp_token_list)) == 1:
            return tok_results
        else:
            if bp_token_list != [] and len(bp_token_list) > 1:
                while bp_token_list[-1] == '':
                    bp_token_list.pop()

        tester = []
        for i in range(len(token_list)-len(bp_token_list)):
            # Not looking at the first and last because that was causing mis-match issues
            if token_list[i:i+len(bp_token_list)] == bp_token_list:
                tester.append(i)
                tester.append(i+len(bp_token_list))


        if tester == []:

            bp_token_list[0] = 'Ġ'+bp_token_list[0]
            tester2 = []
            for i in range(len(token_list)-len(bp_token_list)):
                if token_list[i:i+len(bp_token_list)] == bp_token_list:
                    tester2.append(i)
                    tester2.append(i+len(bp_token_list))
            if tester2 == []:
                tester3 = []
                for i in range(len(token_list)-len(bp_token_list)):
                    if token_list[i+1:i+len(bp_token_list)] == bp_token_list[1:]:
                        tester3.append(i)
                        tester3.append(i+len(bp_token_list))
                if tester3 == []:
                    return tok_results
                else:
                    ner_tags = tag_vulparts_v2(token_list, mode, tester3, token_dict)
            else:
              ner_tags = tag_vulparts_v2(token_list, mode, tester2, token_dict)
        else:
            ner_tags = tag_vulparts_v2(token_list, mode, tester, token_dict)

        token_list.append('</s>')
        token_list.insert(0, '<s>')
        ner_tags.append(-100)
        ner_tags.insert(0, -100)
        tok_results.append({"ner_tags": ner_tags, 'tokens': token_list,
                            'input_ids':input_ids, 'attention_mask':attention_mask})

    else:
        token_list.append('</s>')
        token_list.insert(0, '<s>')
        ner_tags = [0]*len(token_list)
        tok_results.append({"ner_tags": ner_tags, 'tokens': token_list,
                            'input_ids':input_ids, 'attention_mask':attention_mask})

    return tok_results

In [ ]:
def shorten_data_with_windows_after_tokenization(infodict):
    '''
    Shortening after tokenization
    '''
    new_info = []
    new_size = 510

    tags = infodict['ner_tags']
    tokens = infodict['tokens']
    attn = infodict['attention_mask']
    input_ids = infodict['input_ids']
    tag_indeces = [i for i, x in enumerate(tags) if x != 0 and x != -100]

    # If there are more than 512 tokens
    if len(tokens) >= new_size:
        # If the indeces start and stop before 512, cut the list to 512
        if tag_indeces[0] < new_size and tag_indeces[-1] <= new_size:
            breaks = [i for i, x in enumerate(tokens) if '\n' in x or ')' in x or ']' in x or '}' in x or '\t' in x]
            if [m for m in breaks if m <= new_size] == []:
                new_info.append({'new_tokens': [], 'new_tags': [],
                            'new_attn': [], 'new_input_ids': []})
                return new_info
            else:
                break_id_to_use = [m for m in breaks if m <= new_size][-1]
                new_tokens = tokens[:break_id_to_use]
                new_tags = tags[:break_id_to_use]
                new_attn = attn[:break_id_to_use]
                new_input_ids = input_ids[:break_id_to_use]
                new_info.append({'new_tokens': new_tokens, 'new_tags': new_tags,
                                'new_attn': new_attn, 'new_input_ids': new_input_ids})

        elif len(tag_indeces) <= new_size:
            breaks = [i for i, x in enumerate(tokens) if '\n' in x or ')' in x or ']' in x or '}' in x]
            if [m for m in breaks if m <= tag_indeces[0]] == []:
                new_info.append({'new_tokens': [], 'new_tags': [],
                            'new_attn': [], 'new_input_ids': []})
                return new_info
            else:
                break_id_to_use1 = [m for m in breaks if m <= tag_indeces[0]][-1]
                break_id_to_use2 = [m for m in breaks if m >= break_id_to_use1 and m <= break_id_to_use1+new_size][-1]
                new_tokens = tokens[break_id_to_use1:break_id_to_use2]
                new_tags = tags[break_id_to_use1:break_id_to_use2]
                new_attn = attn[break_id_to_use1:break_id_to_use2]
                new_input_ids = input_ids[break_id_to_use1:break_id_to_use2]
                new_info.append({'new_tokens': new_tokens, 'new_tags':new_tags,
                                'new_attn': new_attn, 'new_input_ids': new_input_ids})
        else:
            breaks = [i for i, x in enumerate(tokens) if '\n' in x or ')' in x or ']' in x or '}' in x]
            break_ids_to_use = []
            for i in range(math.ceil(len(tokens)/new_size)):
                if i == 0:
                    if [m for m in breaks if m <=new_size] == []:
                        break_id = breaks[0]
                    else:
                        break_id = [m for m in breaks if m <=new_size][-1]
                    chunk_start = 0
                else:
                    break_id = [m for m in breaks if m >= break_ids_to_use[i-1] and m <= break_ids_to_use[i-1]+new_size][-1]
                    chunk_start = break_ids_to_use[i-1]
                break_ids_to_use.append(break_id)
                chunk = [chunk_start, break_id]
                new_tokens = tokens[chunk[0]:chunk[1]]
                new_tags = tags[chunk[0]:chunk[1]]
                new_attn = attn[chunk[0]:chunk[1]]
                new_input_ids = input_ids[chunk[0]:chunk[1]]
                new_info.append({'new_tokens': new_tokens, 'new_tags':new_tags,
                                'new_attn': new_attn, 'new_input_ids': new_input_ids})

    else:
        ## If there are not more than 512 tokens
        new_tokens = tokens
        new_tags = tags
        new_attn = attn
        new_input_ids = input_ids
        new_info.append({'new_tokens': new_tokens, 'new_tags':new_tags,
                        'new_attn': new_attn, 'new_input_ids': new_input_ids})

    return new_info

In [ ]:
def create_labels(row):
    tags = row['ner_tags']
    labels = []
    to_change = tags
    for x in to_change:
        x[0] = -100
        x[-1] = -100
        pad = 512 - len(x)
        x += [0]*pad
        labels.append(x)
    return {'labels':labels}

def change_attention(row):
    attention = row['attention_mask']
    atts = []
    to_change = attention
    for x in to_change:
        if 0 in x:
            indx_to_change = x.index(0)
            x[indx_to_change-1] = 0
        else:
            x[-1] = 0
        x[0] = 0
        atts.append(x)
    return {'attention_mask':atts}

In [ ]:
def DistilBERT_tokenizer_tag(row):
    mode = row.cwetype
    content = row.short_text

    tok_results = []

    token_dict = tokenizer(content.strip(), truncation=True, max_length = 512,  padding='max_length')
    input_ids = token_dict['input_ids']
    attention_mask = token_dict['attention_mask']
    tokens = tokenizer.convert_ids_to_tokens(token_dict.input_ids)

    token_list = [tok for tok in tokens if tok != '[SEP]' and tok != '[PAD]' and tok != '[CLS]']

    if row.type == 'bp':
        bp = row.parts

        bp_token_dict = tokenizer(bp.strip(), max_length = 512, truncation = True)
        bp_tokens = tokenizer.convert_ids_to_tokens(bp_token_dict.input_ids)
        bp_token_list = [bp_tok for bp_tok in bp_tokens if bp_tok != '[SEP]' and bp_tok != '[PAD]' and bp_tok != '[CLS]']

        if bp_token_list == [] or len(set(bp_token_list)) == 1:
            return tok_results
        else:
            if bp_token_list != [] and len(bp_token_list) > 1:
                while bp_token_list[-1] == '':
                    bp_token_list.pop()

        tester = []
        for i in range(len(token_list)-len(bp_token_list)):
            # Not looking at the first and last because that was causing mis-match issues
            if token_list[i:i+len(bp_token_list)] == bp_token_list:
                tester.append(i)
                tester.append(i+len(bp_token_list))

        if tester == []:

            tester3 = []
            for i in range(len(token_list)-len(bp_token_list)):
                if token_list[i+1:i+len(bp_token_list)+1] == bp_token_list:
                    tester3.append(i)
                    tester3.append(i+len(bp_token_list))
            if tester3 == []:
                return tok_results
            else:
                ner_tags = tag_vulparts(token_list, mode, tester3)
        else:
            ner_tags = tag_vulparts(token_list, mode, tester)

        tok_results.append({"ner_tags": ner_tags, 'tokens': token_list,
                            'input_ids':input_ids, 'attention_mask':attention_mask})

    else:
        ner_tags = [0]*len(token_list)
        tok_results.append({"ner_tags": ner_tags, 'tokens': token_list,
                            'input_ids':input_ids, 'attention_mask':attention_mask})

    return tok_results

## Import Collected Data

In [ ]:
file_list = list(os.listdir('Data\\'))

all_data = []

for file in file_list:
    with open('Data\\'+file, 'r') as infile:
        data = json.load(infile)
    if len(data) != 0:
        print(file, len(data))
        for info in data:
            all_data.append(info)
print(len(all_data))

In [ ]:
labeled_dict_list = []
step1_dict_list = []
toolongdict = dict()

slightly_cleaned_all_data = [x for x in all_data if '<html' not in x['orig_txt']]
slightly_cleaned_all_data = [x for x in slightly_cleaned_all_data if 'Search.setIndex' not in x['orig_txt']]
slightly_cleaned_all_data = [x for x in slightly_cleaned_all_data if 'JavaScript Library' not in x['orig_txt']]

## Cleaning

In [ ]:
df_all = pd.DataFrame(slightly_cleaned_all_data)
df_new = df_all.copy()
df_new = df_new[['cwetype', 'commit', 'neutralparts', 'vulparts','orig_diff','orig_txt']]
df_new.head(5)

In [ ]:
%%time
neutraltext = [getneutralText(x) for x in df_new.orig_diff]

In [ ]:
## VUL PARTS ##
df_bp = df_new.copy()
df_bp = df_bp[['cwetype', 'vulparts', 'orig_txt']]

# find all single comments and replace with nothing
bp_text_without_single_comments = [re.sub('(#.*)', '', str(x)) for x in df_bp.orig_txt.tolist()]
# find all comment blocks and replace with nothing - double quotes
bp_text_without_comment_blocks1 = [re.sub(r'(["])\1\1(.*?)\1{3}', '', str(x), flags = re.DOTALL) for x in bp_text_without_single_comments]
# find all comment blocks and replace with nothing - single quotes
bp_text_without_comment_blocks2 = [re.sub(r"(['])\1\1(.*?)\1{3}", ' ', str(x), flags = re.DOTALL) for x in bp_text_without_comment_blocks1]

df_bp['text'] = bp_text_without_comment_blocks2
df_bp = df_bp.drop(columns =['orig_txt'])
df_bp['type'] = ['bp']*len(df_bp)
df_bp.head(2)

In [ ]:
df_bp = df_bp.rename(columns = {'vulparts':'parts'})
df = df_bp.copy()
df.head(10)

In [ ]:
#saving just in case
import pickle
with open('clean_dataset.pickle', 'wb') as output:
    pickle.dump(df, output)

In [ ]:
import pickle
with open('clean_dataset.pickle', 'rb') as input:
    df = pickle.load(input)
df.head()

In [ ]:
df = df.explode('parts')
df = df.fillna('')
df = df[df.text != '']
df = df[df.parts != '']
df = df[df.parts.str.len() > 1]
df['len_set_parts'] = df['parts'].apply(lambda x: len(set(x.strip())))
df = df[df.len_set_parts > 3 ]

In [ ]:
cwe = []
for x in zip(df.type.tolist(), df.cwetype.tolist()):
  if x[0] == 'gp':
    cwe.append('neutral')
  else:
    cwe.append(x[1])
df['cwetype'] = cwe
df.cwetype.value_counts()

In [ ]:
df['bp_len'] = [len(x) for x in df.parts]

In [ ]:
def find_partstart(row):
    if row.type == 'bp':
        bp_index = row['text'].find(row['parts'])
    else:
        bp_index = ''
    return bp_index

In [ ]:
%%time
df['bp_index'] = df.apply(find_partstart, axis =1)

In [ ]:
%%time
results = []
for x in range(0, len(df), 50000):
  print(len(df)-x)
  df_test = df.copy()
  df_test = df_test.iloc[x: x+50000]
  results.append(df_test.apply(shorten_data_with_windows_v2, axis =1))

In [ ]:
short_results = [i for info in results for i in info]
len(short_results)

In [ ]:
df['short_text'] = short_results
print(len(df))

In [ ]:
df = df.explode('short_text')
print(len(df))
df.head()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df = df[df.short_text !='']
df = df.dropna(subset=['short_text'])
df = df[df.parts != '']
df = df[df.parts.str.len() >= 3]
df = df.reset_index(drop=True)
print(len(df))

In [ ]:
df['short_text2'] = [x.strip() for x in df.short_text.to_list()]

In [ ]:
df = df.drop(columns = ['short_text'])
df = df.rename(columns = {'short_text2':'short_text'})

In [ ]:
df.head()

In [ ]:
#saving just in case
import pickle
with open('clean_dataset_short.pickle', 'wb') as output:
    pickle.dump(df, output)

In [ ]:
#opening
import pickle
with open('clean_dataset_short.pickle', 'rb') as input:
    df = pickle.load(input)

## CodeBERT OR RoBERTa Tokenizer

In [ ]:
from transformers import AutoTokenizer
# Change here for specific tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
#tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
%%time
df = df[~df.short_text.str.startswith('cdef')]
df = df[~df.parts.str.strip().str.startswith('#')]
df = df.drop_duplicates()
len(df)

In [ ]:
df_bp = df.copy()
df_bp = df_bp[df_bp.type == 'bp']
df_bp.reset_index(inplace=True, drop=True)
len(df_bp)

In [ ]:
%%time
tag_tok_results_bp = []
for x in range(0, len(df_bp), 50000):
  df_test = df_bp.copy()
  df_test = df_test.iloc[x: x+50000]
  tag_tok_results_bp.append(df_test.apply(CODEBERT_tokenizer_tag_v2, axis =1))
  print(len(df_bp)-x)

In [ ]:
tag_tok_results_bp_all = [y for x in tag_tok_results_bp for y in x]
len(tag_tok_results_bp_all)

In [ ]:
bp_ner_tags = []
bp_tokens = []
bp_attention = []
bp_input_ids = []
for x in tag_tok_results_bp_all:
  if len(x) == 0:
    bp_ner_tags.append('None')
    bp_tokens.append('None')
    bp_attention.append('None')
    bp_input_ids.append('None')
  else:
    for y in x:
      bp_ner_tags.append(y['ner_tags'])
      bp_tokens.append(y['tokens'])
      bp_attention.append(y['attention_mask'])
      bp_input_ids.append(y['input_ids'])

In [ ]:
len(bp_ner_tags), len(bp_tokens), len(bp_attention), len(bp_input_ids)

In [ ]:
df_bp['ner_tags'] = bp_ner_tags
df_bp['tokens'] = bp_tokens
df_bp['attention_mask'] = bp_attention
df_bp['input_ids'] = bp_input_ids

df_bp = df_bp.drop(columns = ['bp_len'])

In [ ]:
df_bp = df_bp[df_bp.ner_tags != 'None']
df_bp.reset_index(drop=True, inplace=True)
df_bp.head(2)

In [ ]:
#saving just in case
import pickle
with open('dataset_short_tag_tok_bp.pickle', 'wb') as output:
    pickle.dump(df_bp, output)

In [ ]:
import pickle
with open('dataset_short_tag_tok_bp.pickle', 'rb') as input:
    df_bp = pickle.load(input)

In [ ]:
df_bp = df_bp.drop(columns = ['text'])
df_bp = df_bp[['cwetype', 'type', 'short_text', 'parts', 'ner_tags', 'tokens', 'attention_mask','input_ids']]

In [ ]:
%%time
results2 = []
for x in range(0, len(df_bp), 100000):
  print(len(df_bp)-x)
  df_test = df_bp.copy()
  df_test = df_test.iloc[x: x+100000]
  results2.append(df_test.apply(shorten_data_with_windows_after_tokenization, axis =1))

In [ ]:
short_results2 = [i for info in results2 for i in info]
len(short_results2)

In [ ]:
df_new = df_bp.copy()
df_new['short_results'] = short_results2

df_new = df_new.explode('short_results')
print(len(df_new))
df_new.head()

In [ ]:
new_ner_tags = []
new_tokens = []
new_attention = []
new_input_ids = []
for x in df_new.short_results.to_list():
    if x['new_tags'] == []:
        new_ner_tags.append('None')
        new_tokens.append('None')
        new_attention.append('None')
        new_input_ids.append('None')
    else:
        if len(x['new_tags']) == len(x['new_tokens']) == len(x['new_attn']) == len(x['new_input_ids']):
            new_ner_tags.append(x['new_tags'])
            new_tokens.append(x['new_tokens'])
            new_attention.append(x['new_attn'])
            new_input_ids.append(x['new_input_ids'])
        else:
            new_ner_tags.append('None')
            new_tokens.append('None')
            new_attention.append('None')
            new_input_ids.append('None')

In [ ]:
df_new['new_tags'] = new_ner_tags
df_new['new_tokens'] = new_tokens
df_new['new_attn'] = new_attention
df_new['new_input_ids'] = new_input_ids

In [ ]:
df_new.head(1)

In [ ]:
df_new = df_new[df_new.new_tokens != 'None']
df_new = df_new.reset_index(drop=True)

print(len(df_new))

In [ ]:
df = df.drop(columns = ['short_text'])
df = df.rename(columns = {'short_text2':'short_text'})

In [ ]:
df_new.cwetype.value_counts()

In [ ]:
df_new['len_set_parts'] = df_new['parts'].apply(lambda x: len(set(x)))

In [ ]:
df_bp_50 = df_new.copy()
df_bp_50 = df_bp_50[df_bp_50.parts.str.len() >= 50]

In [ ]:
df_bp_50['parts_stripped'] = df_bp_50['parts'].apply(lambda x: x.strip())
df_bp_50['len_set_parts'] = df_bp_50['parts_stripped'].apply(lambda x: len(set(x)))

In [ ]:
len(df_bp_50[df_bp_50.len_set_parts > 5])

In [ ]:
df_bp_50 = df_bp_50[df_bp_50.len_set_parts >= 5]

In [ ]:
print(df_bp_50.cwetype.value_counts())

In [ ]:
df_bp_50['len_set_toks'] = df_bp_50['new_tokens'].apply(lambda x: len(x))
len(df_bp_50[df_bp_50.len_set_toks > 5])

In [ ]:
from sklearn.utils.random import sample_without_replacement
neutrals_to_take = sample_without_replacement(len(df_gp), 16480, random_state = 2023)
neutrals_to_take.sort()
neutrals_to_take = neutrals_to_take.tolist()
len(df_gp.iloc[neutrals_to_take])
df_gp_16480 = df_gp.iloc[neutrals_to_take]

In [ ]:
df_final = df_bp_50.copy()
print(len(df_final))
print(df_final.cwetype.value_counts())
df_final.head(2)

In [ ]:
#saving just in case
import pickle
with open('df_tagsandtokens.pickle', 'wb') as output:
    pickle.dump(df_final, output)

In [ ]:
df_final = df_final.drop(columns = ['ner_len','len_set_parts', 'parts_stripped'])

In [ ]:
print(df_final.cwetype.value_counts())

In [ ]:
len(df_final)

In [ ]:
from collections import Counter

count = Counter()

for row in df_final.new_tags.tolist():
  for item in row:
    count[item] += 1

In [ ]:
count.most_common(13)

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split

In [ ]:
%%time
train, test = train_test_split(df_final, test_size=0.4, random_state=2023, shuffle = True, stratify = df_final.cwetype)

In [ ]:
import datasets
print('train counts:\n', train.cwetype.value_counts())
train_data = datasets.Dataset.from_pandas(train)
print('len of train data', len(train_data))
print()

valid, test2 = train_test_split(test, test_size=0.5, random_state=2023, shuffle = True, stratify = test.cwetype)
print('test counts:\n',test2.cwetype.value_counts())
test_data = datasets.Dataset.from_pandas(test2)
print('len of test data', len(test_data))
print()

print('valid counts:\n', valid.cwetype.value_counts())
valid_data = datasets.Dataset.from_pandas(valid)
print('len of valid data', len(valid_data))
print()

dataset = datasets.DatasetDict({"train":train_data,'validation':valid_data, 'test':test_data})
dataset

In [ ]:
dataset_labels = dataset.map(create_labels, batched=True)

In [ ]:
dataset_v2 = dataset_labels.map(change_attention, batched=True)

In [ ]:
dataset_v2

In [ ]:
import pickle
with open('PADDED_Roberta.pickle', 'wb') as output:
    pickle.dump(dataset_v2, output)

In [ ]:
with open('dataset_tagsandtokens.pickle', 'rb') as input:
    dataset = pickle.load(input)

## DistilBERT Tokenizer

In [ ]:
import datasets
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
%%time
df = df[~df.short_text.str.startswith('cdef')]
df = df[~df.parts.str.strip().str.startswith('#')]
df = df.drop_duplicates()
len(df)

In [ ]:
df_bp = df.copy()
df_bp = df_bp[df_bp.type == 'bp']
df_bp.reset_index(inplace=True, drop=True)
len(df_bp)

In [ ]:
df_bp.cwetype.value_counts()

In [ ]:
%%time
tag_tok_results_bp = []
for x in range(0, len(df_bp), 25000):
  df_test = df_bp.copy()
  df_test = df_test.iloc[x: x+25000]
  tag_tok_results_bp.append(df_test.apply(DistilBERT_tokenizer_tag, axis =1))
  print(len(df_bp)-x)

In [ ]:
tag_tok_results_bp_all = [y for x in tag_tok_results_bp for y in x]
len(tag_tok_results_bp_all)

In [ ]:
bp_ner_tags = []
bp_tokens = []
bp_attention = []
bp_input_ids = []
for x in tag_tok_results_bp_all:
  if len(x) == 0:
    bp_ner_tags.append('None')
    bp_tokens.append('None')
    bp_attention.append('None')
    bp_input_ids.append('None')
  else:
    for y in x:
      bp_ner_tags.append(y['ner_tags'])
      bp_tokens.append(y['tokens'])
      bp_attention.append(y['attention_mask'])
      bp_input_ids.append(y['input_ids'])

In [ ]:
len(bp_ner_tags), len(bp_tokens), len(bp_attention), len(bp_input_ids)

In [ ]:
df_bp['ner_tags'] = bp_ner_tags
df_bp['tokens'] = bp_tokens
df_bp['attention_mask'] = bp_attention
df_bp['input_ids'] = bp_input_ids
df_bp = df_bp.drop(columns = ['bp_len'])
df_bp = df_bp[df_bp.ner_tags != 'None']
df_bp.reset_index(drop=True, inplace=True)
df_bp.head(3)

In [ ]:
#saving just in case
import pickle
with open('dataset_short_tag_tok_bp.pickle', 'wb') as output:
    pickle.dump(df_bp, output)

In [ ]:
with open('dataset_short_tag_tok_bp.pickle', 'rb') as input:
    df_bp = pickle.load(input)

In [ ]:
df_bp = df_bp.drop(columns = ['text'])
df_bp = df_bp[['cwetype', 'type', 'short_text', 'parts', 'ner_tags', 'tokens', 'attention_mask','input_ids']]
df_bp.head()

In [ ]:
df_bp.cwetype.value_counts()

In [ ]:
df_bp['len_set_parts'] = df_bp['parts'].apply(lambda x: len(set(x)))

In [ ]:
df_bp_50 = df_bp.copy()
df_bp_50 = df_bp_50[df_bp_50.parts.str.len() >= 50]

In [ ]:
df_bp_50['parts_stripped'] = df_bp_50['parts'].apply(lambda x: x.strip())
df_bp_50['len_set_parts'] = df_bp_50['parts_stripped'].apply(lambda x: len(set(x)))

In [ ]:
len(df_bp_50[df_bp_50.len_set_parts > 5])

In [ ]:
df_bp_50 = df_bp_50[df_bp_50.len_set_parts >= 5]

In [ ]:
print(df_bp_50.cwetype.value_counts())

In [ ]:
from sklearn.utils.random import sample_without_replacement
neutrals_to_take = sample_without_replacement(len(df_gp), 16480, random_state = 2023)
neutrals_to_take.sort()
neutrals_to_take = neutrals_to_take.tolist()
len(df_gp.iloc[neutrals_to_take])
df_gp_16480 = df_gp.iloc[neutrals_to_take]

In [ ]:
df_final = df_bp_50.copy()
print(len(df_final))
print(df_final.cwetype.value_counts())
df_final.head(2)

In [ ]:
#saving just in case
import pickle
with open('df_tagsandtokens.pickle', 'wb') as output:
    pickle.dump(df_final, output)

In [ ]:
df_final['ner_len'] = [len(x) for x in df_final.ner_tags]
df_final = df_final[df_final.short_text != '']

In [ ]:
df_final = df_final.drop(columns = ['ner_len','len_set_parts', 'parts_stripped'])

In [ ]:
df_final.head(1)

In [ ]:
print(df_final.cwetype.value_counts())

In [ ]:
from collections import Counter

count = Counter()

for row in df_final.ner_tags.tolist():
  for item in row:
    count[item] += 1

In [ ]:
count.most_common(13)

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split

In [ ]:
%%time
train, test = train_test_split(df_final, test_size=0.4, random_state=2023, shuffle = True, stratify = df_final.cwetype)

In [ ]:
import datasets
print('train counts:\n', train.cwetype.value_counts())
train_data = datasets.Dataset.from_pandas(train)
print('len of train data', len(train_data))
print()

valid, test2 = train_test_split(test, test_size=0.5, random_state=2023, shuffle = True, stratify = test.cwetype)
print('test counts:\n',test2.cwetype.value_counts())
test_data = datasets.Dataset.from_pandas(test2)
print('len of test data', len(test_data))
print()

print('valid counts:\n', valid.cwetype.value_counts())
valid_data = datasets.Dataset.from_pandas(valid)
print('len of valid data', len(valid_data))
print()

dataset = datasets.DatasetDict({"train":train_data,'validation':valid_data, 'test':test_data})
dataset

In [ ]:
dataset_labels = dataset.map(create_labels, batched=True)

In [ ]:
dataset_v2 = dataset_labels.map(change_attention, batched=True)

In [ ]:
dataset_v2

In [ ]:
import pickle
with open('PADDED_distilbert.pickle', 'wb') as output:
    pickle.dump(dataset_v2, output)

In [ ]:
with open('dataset_tagsandtokens.pickle', 'rb') as input:
    dataset = pickle.load(input)

# Model Development

In [ ]:
!pip install datasets transformers evaluate seqeval pynvml pyyaml h5py huggingface_hub

In [ ]:
import pandas as pd
import os
import json
import random
import datasets
import pickle
from datasets import load_metric
import evaluate
import numpy as np
from pynvml import *
import tensorflow as tf

import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdmn = tqdm.notebook.tqdm

import transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import AutoModelForTokenClassification, AutoTokenizer

import sklearn
from sklearn.dummy import DummyClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForTokenClassification,
   AdamW,
   AdamWeightDecay)

import torch
import torch.nn as nn
import torch.optim as optim

from datasets import load_dataset

In [ ]:
#file = 'roberta_tagsandtokens_PADDED.pickle'
file = 'distilbert_tagsandtokens_PADDED.pickle'
#file = 'codebert_tagsandtokens_PADDED.pickle'
with open(file, 'rb') as data:
    dataset = pickle.load(data)
dataset

In [ ]:
model_choice = "roberta-base"
tok_choice = "roberta-base"
#model_choice = 'microsoft/codebert-base'
#tok_choice = 'microsoft/codebert-base'
#model_choice = "distilbert-base-uncased"
#tok_choice = "distilbert-base-uncased"

In [ ]:
label_list = ['O',      #0
              'B-rce',  #1
              'I-rce',  #2
              'B-oob',  #3
              'I-oob',  #4
              'B-xss',  #5
              'I-xss',  #6
              'B-sql',  #7
              'I-sql',  #8
              'B-iiv',  #9
              'I-iiv',  #10
              'B-pat',  #11
              'I-pat',  #12
             ]
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

In [ ]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_choice, num_labels = len(label_list))

model.config.id2label = id2label
model.config.label2id = label2id

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
optimizer = optim.AdamW(params=model.parameters(), lr=2e-5)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)

In [ ]:
from torch.utils.data import DataLoader

train_data = DataLoader(
    dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_data = DataLoader(
    dataset["validation"], batch_size=8, collate_fn=data_collator
)

In [ ]:
from transformers import get_scheduler

num_epochs = 10
num_training_steps = num_epochs * len(train_data)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

In [ ]:
train_loss = []
model.train()
for epoch in tqdmn(range(num_epochs)):
    current_loss = 0
    for i, batch in enumerate(tqdmn(train_data)):
        # move the batch tensors to the same device as the
        batch = { k: v.to(device) for k, v in batch.items() }
        # send 'input_ids', 'attention_mask' and 'labels' to the model
        outputs = model(**batch)
        loss = outputs.loss #outputs[0]
        loss.backward()
        current_loss += loss.item()
        optimizer.step()
        lr_scheduler.step()

        optimizer.zero_grad()
        train_loss.append(current_loss / 10)
        current_loss = 0

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import login

notebook_login()

In [ ]:
model.save_pretrained('PyNERModel_dev')

In [ ]:
from huggingface_hub import whoami
whoami()

In [ ]:
model.push_to_hub(repo_id = '') #need to input user id

# Model Test

In [ ]:
!pip install huggingface_hub datasets transformers

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import login

notebook_login()

In [ ]:
from transformers import AutoTokenizer
#tok_choice = 'roberta-base'
tok_choice = 'microsoft/codebert-base'
#tok_choice = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(tok_choice)

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained('mmeberg/CoCo_PyVulDet_NER')
#'mmeberg/CoCo_PyVulDet_NER'
#'mmeberg/RoCo_PyVulDet_NER'
#'mmeberg/RoRo_PyVulDet_NER'
#'mmeberg/CoRo_PyVulDet_NER'
#'mmeberg/DiDi_PyVulDet_NER'

In [ ]:
import tqdm
tqdmn = tqdm.notebook.tqdm

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)

In [ ]:
model.config

## GitHub Test Set

In [ ]:
import pickle
import datasets

file = 'codebert_tagsandtokens_PADDED.pickle'
with open(file, 'rb') as data:
    dataset = pickle.load(data)
dataset

In [ ]:
import pandas as pd
test_df = pd.DataFrame(dataset['test'])
test_df = test_df[['input_ids', 'attention_mask', 'labels']]
test_df.head(2)

In [ ]:
test_data = datasets.Dataset.from_pandas(test_df)

In [ ]:
test_data.set_format(type='torch')

In [ ]:
test_data.format['type']

In [ ]:
from torch.utils.data import DataLoader

test_data = DataLoader(
    test_data, shuffle=True, batch_size=8, collate_fn=data_collator
)

In [ ]:
label_list = ['O',      #0
              'B-rce',  #1
              'I-rce',  #2
              'B-oob',  #3
              'I-oob',  #4
              'B-xss',  #5
              'I-xss',  #6
              'B-sql',  #7
              'I-sql',  #8
              'B-iiv',  #9
              'I-iiv',  #10
              'B-pat',  #11
              'I-pat',  #12
             ]
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

In [ ]:
#Expand results so no longer lists of lists
for_cm_true = [y for x in all_true for y in x]
print(len(for_cm_true))
for_cm_pred = [y for x in all_pred for y in x]
print(len(for_cm_pred))

In [ ]:
labels_name = list(label2id.keys())
print(labels_name)
labels_id = list(id2label.keys())
print(labels_id)

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
import sklearn.metrics as skm
import seaborn as sns

In [ ]:
labels_id_11 = list(id2label.keys())[1:]
print(labels_id_11)
labels_name_11 = list(label2id.keys())[1:]
print(labels_name_11)

In [ ]:
cm = confusion_matrix(for_cm_true, for_cm_pred, labels =labels_id_11)

In [ ]:
# Normalize
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=labels_name_11, yticklabels=labels_name_11)
plt.ylabel('Actual')
plt.xlabel('Predicted')
#plt.savefig('plot4.jpg', format='jpg', dpi=1200)
plt.show(block=False)

In [ ]:
import numpy as np
confusion_np = cmn#.np()
FP = confusion_np.sum(axis=0) - np.diag(confusion_np)
FN = confusion_np.sum(axis=1) - np.diag(confusion_np)
TP = np.diag(confusion_np)
TN = confusion_np.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP)
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
#F1-Score
F1 = TP/(TP+.5*(FP+FN))

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

metrics = {'FP':FP, 'FN':FN, 'TP':TP, 'TN':TN, 'TPR':TPR, 'TNR':TNR, 'PPV':PPV, 'NPV':NPV, 'FPR':FPR, 'FNR':FNR, 'FDR':FDR, 'F1':F1, 'ACC':ACC}
for k, v in metrics.items():
    print(k)
    for x in zip(labels_name_11, list(v)):
        print(x[0], x[1])
    print()

In [ ]:
test_tns = [
 'rce', #1
 'oob', #2
 'xss', #3
 'sql', #4
 'iiv', #5
 'pat'  #6
 ]

test_tns_ids = [ 1, 2, 3, 4, 5, 6]

cwes = [
 'CWE-94', #1
 'CWE-787', #2
 'CWE-79', #3
 'CWE-89', #4
 'CWE-20', #5
 'CWE-22'  #6
 ]


id2label_test = {i: label for i, label in enumerate(test_tns,1)}
label2id_test = {label: i for i, label in enumerate(test_tns,1)}

all_true_test = [label2id_test[id2label[x].split('-')[1]] if x!=0 else 0 for x in for_cm_true]
all_pred_test = [label2id_test[id2label[x].split('-')[1]] if x!=0 else 0 for x in for_cm_pred]

#print(classification_report(all_true_test, all_pred_test, target_names=test_tns))

In [ ]:
cm7 = confusion_matrix(all_true_test, all_pred_test, labels=test_tns_ids)
cmn7 = cm7.astype('float') / cm7.sum(axis=1)[:, np.newaxis]

In [ ]:
confusion7_np = cmn7
FP = confusion7_np.sum(axis=0) - np.diag(confusion7_np)
FN = confusion7_np.sum(axis=1) - np.diag(confusion7_np)
TP = np.diag(confusion7_np)
TN = confusion7_np.sum() - (FP + FN + TP)


# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP)
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
#F1-Score
F1 = TP/(TP+.5*(FP+FN))

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

metrics = {'FP':FP, 'FN':FN, 'TP':TP, 'TN':TN, 'TPR':TPR, 'TNR':TNR, 'PPV':PPV, 'NPV':NPV, 'FPR':FPR, 'FNR':FNR, 'FDR':FDR, 'F1':F1, 'ACC':ACC}
for k, v in metrics.items():
    print(k)
    for x in zip(test_tns, list(v)):
        print(x[0], x[1])
    print()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(cmn7, annot=True, fmt='.2f', xticklabels=cwes, yticklabels=cwes)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))

plt.subplot(1, 2, 1)
sns1 = sns.heatmap(cmn7, annot=True, fmt='.2f', ax=ax1, xticklabels=cwes, yticklabels=cwes, annot_kws={"size": 15})
plt.ylabel('Actual', fontsize=15)
plt.xlabel('Predicted', fontsize=15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
cbar1 = sns1.collections[0].colorbar
cbar1.ax.tick_params(labelsize=14)


plt.subplot(1, 2, 2)
sns2 = sns.heatmap(cmn, annot=True, fmt='.2f', ax=ax2, xticklabels=labels_name_11, yticklabels=labels_name_11, annot_kws={"size": 14})
plt.ylabel('Actual', fontsize=15)
plt.xlabel('Predicted', fontsize=15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
cbar2 = sns2.collections[0].colorbar
cbar2.ax.tick_params(labelsize=14)

plt.tight_layout()
plt.savefig('Figure_A1.jpg', format='jpg', dpi=1200)
plt.show()

## Displacy

In [ ]:
import random
import spacy
import spacy
from spacy import displacy
from transformers import pipeline
from transformers import AutoModelForTokenClassification

In [ ]:
model_for_pipeline = AutoModelForTokenClassification.from_pretrained("mmeberg/CoCo_PyVulDet_NER")
token_classifier = pipeline("ner", model = model_for_pipeline, tokenizer = tokenizer)

In [ ]:
def get_entities_html(text, ner_result, title=None):
    """Visualize NER with the help of SpaCy"""
    ents = []
    for ent in ner_result:
        e = {}
        e["start"] = ent["start"]
        e["end"] = ent["end"]
        # add the score if you want in the label
        #e["label"] = f"{ent["entity"]}-{ent['score']:.2f}"
        e["label"] = ent["entity"]
        if ents and -1 <= ent["start"] - ents[-1]["end"] <= 1 and ents[-1]["label"] == e["label"]:
            ents[-1]["end"] = e["end"]
            continue
        ents.append(e)
    render_data = [
    {
        "text": text,
        "ents": ents,
        "title": title,
    }
    ]
    spacy.displacy.render(render_data, style="ent", manual=True, jupyter=True)

In [ ]:
print(id2label)
print(label2id)

In [ ]:
t = 0
while t != 4:
    i = random.randint(0, len(dataset['test']))
    if int(dataset['test'][i]['labels'][1]) == 0:
        t = 4
        print(i)

print(i)
print(dataset['test'][i]['labels'][:200])
print(dataset['test']['cwetype'][i])


In [ ]:
print(dataset['test']['short_text'][i])

In [ ]:
print(dataset['test']['parts'][i])

In [ ]:
doc_ner = token_classifier(text_to_test)
get_entities_html(text_to_test, doc_ner)